In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.utils import resample

# Wyłączenie ostrzeżeń, które mogą się pojawiać przy wczytywaniu
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn')

C:\Users\Ai\AppData\Local\miniconda3\envs\projekt_stan\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
# --- WCZYTYWANIE DANYCH ---
def read_robust_csv_no_header(filepath, header):
    import csv
    print(f"Wczytuję plik: {filepath}")
    data_rows = []
    malformed_count = 0
    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
        reader = csv.reader(f)
        expected_col_count = len(header)
        for i, row in enumerate(reader, start=1):
            if len(row) == expected_col_count:
                data_rows.append(row)
            else:
                malformed_count += 1
    if malformed_count > 0:
        print(f"OSTRZEŻENIE: Pominięto łącznie {malformed_count} uszkodzonych wierszy.")
    return pd.DataFrame(data_rows, columns=header)

CORRECT_HEADER = ['SaleId', 'OriginalId', 'PortalId', 'Title', 'Description', 'Area', 'Price', 'OfferPrice', 'RealPriceAfterRenovation', 'OriginalPrice', 'PricePerSquareMeter', 'NumberOfRooms', 'BuiltYear', 'Type', 'BuildingType', 'BuildingCondition', 'OfferFrom', 'Floor', 'Floors', 'TypeOfMarket', 'OwnerType', 'DateAddedToDatabase', 'DateAdded', 'DateLastModification', 'DateLastRaises', 'NewestDate', 'AvailableFrom', 'Link', 'Phone', 'MainImage', 'OtherImages', 'NumberOfDuplicates', 'NumberOfRaises', 'NumberOfModifications', 'IsDuplicatePriceLower', 'IsDuplicatePrivateOwner', 'Score', 'ScorePrecision', 'CommunityScore', 'NumberOfCommunityComments', 'NumberOfCommunityOpinions', 'Archive', 'Location', 'VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber', 'RegionNumber', 'SubRegionNumber', 'StreetNumber', 'EncryptedId', 'PredictedRenovation', 'LocationPath']
if len(CORRECT_HEADER) != 53:
    CORRECT_HEADER.extend([f'UnknownCol_{i}' for i in range(53 - len(CORRECT_HEADER))])

df_added = read_robust_csv_no_header('saleflats_2024_dateAdded_polska.csv', header=CORRECT_HEADER)
df_newest = read_robust_csv_no_header('saleflats_2024_newestDate_polska.csv', header=CORRECT_HEADER)

for df_temp in [df_added, df_newest]:
    df_temp['SaleId'] = pd.to_numeric(df_temp['SaleId'], errors='coerce')
df_added.dropna(subset=['SaleId'], inplace=True)
df_newest.dropna(subset=['SaleId'], inplace=True)
for df_temp in [df_added, df_newest]:
    df_temp['SaleId'] = df_temp['SaleId'].astype(int)

df = pd.merge(df_added, df_newest, on='SaleId', how='outer', suffixes=('_added', '_newest'))
for col in df.columns:
    if col.endswith('_added'):
        base_col_name = col.replace('_added', '')
        newest_col_name = f"{base_col_name}_newest"
        if newest_col_name in df.columns:
            df[col] = df[col].fillna(df[newest_col_name])
            df.rename(columns={col: base_col_name}, inplace=True)
            df.drop(columns=[newest_col_name], inplace=True)

print(f"Dane wczytane i połączone. Kształt: {df.shape}")

Wczytuję plik: saleflats_2024_dateAdded_polska.csv
OSTRZEŻENIE: Pominięto łącznie 73512 uszkodzonych wierszy.
Wczytuję plik: saleflats_2024_newestDate_polska.csv
OSTRZEŻENIE: Pominięto łącznie 69269 uszkodzonych wierszy.
Dane wczytane i połączone. Kształt: (760765, 53)


In [3]:
# --- KROK 1: PRZYGOTOWANIE DANYCH ---
print("--- Krok 1: Przygotowanie danych (typy i reguły) ---")
data_to_process = df.copy()
data_to_process.replace(['NULL', ''], np.nan, inplace=True)
data_to_process['BuiltYear'] = pd.to_datetime(data_to_process['BuiltYear'], errors='coerce')
data_to_process.loc[data_to_process['TypeOfMarket'] == 'pierwotny', 'BuildingCondition'] = 'DEVELOPER_STATE'
data_to_process.loc[data_to_process['BuiltYear'].dt.year >= 2024, 'BuildingCondition'] = 'DEVELOPER_STATE'

# Filtrowanie - potrzebujemy tylko opisu i etykiety
data = data_to_process.dropna(subset=['BuildingCondition', 'Description']).copy()
print(f"Liczba wierszy przed balansowaniem: {len(data)}")
print("Rozkład klas PRZED zbalansowaniem:")
print(data['BuildingCondition'].value_counts())

# --- KROK 2: RĘCZNE BALANSOWANIE (UNDERSAMPLING) ---
# Ustawiamy docelową liczbę próbek. Można ją dostosować.
# Weźmy np. 10000, jeśli klasy na to pozwalają, lub mniejszą wartość.
target_samples = 10000 

balanced_dfs = []
for condition_class in data['BuildingCondition'].unique():
    class_df = data[data['BuildingCondition'] == condition_class]
    n_samples = min(len(class_df), target_samples) # Bierzemy max tyle, ile jest lub target
    resampled_df = resample(class_df, 
                            replace=False, 
                            n_samples=n_samples, 
                            random_state=42)
    balanced_dfs.append(resampled_df)

final_data = pd.concat(balanced_dfs)
print("Rozkład klas PO zbalansowaniu:")
print(final_data['BuildingCondition'].value_counts())

# --- KROK 3: UPROSZCZONE PRZYGOTOWANIE DANYCH DO MODELU ---
print("\n--- Krok 3: Uproszczone przygotowanie danych do modelu ---")
numeric_features = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors']
categorical_features = ['BuildingType', 'OfferFrom', 'TypeOfMarket']
# Uzupełnianie braków
for col in numeric_features:
    final_data[col] = pd.to_numeric(final_data[col], errors='coerce').fillna(final_data[col].median())
for col in categorical_features:
    final_data[col] = final_data[col].fillna(final_data[col].mode()[0])
final_data['year'] = pd.to_datetime(final_data['BuiltYear'], errors='coerce').dt.year.fillna(final_data['BuiltYear'].dt.year.median())

# Mapowanie etykiet
labels = final_data['BuildingCondition'].astype('category')
label_mapping = dict(enumerate(labels.cat.categories))
y_categorical = to_categorical(labels.cat.codes)

# Przygotowanie danych tekstowych
max_words, max_len = 10000, 200
tokenizer = Tokenizer(num_words=max_words, oov_token="<UNK>")
tokenizer.fit_on_texts(final_data['Description'])
X_text = pad_sequences(tokenizer.texts_to_sequences(final_data['Description']), maxlen=max_len)

# ---- RĘCZNE PRZYGOTOWANIE DANYCH TABELARYCZNYCH ----
# 1. Skalowanie cech numerycznych
scaler = StandardScaler()
X_numeric = scaler.fit_transform(final_data[numeric_features + ['year']])

# 2. Ręczne kodowanie One-Hot
# Ważne: używamy get_dummies z Pandas, jest prostszy
X_categorical = pd.get_dummies(final_data[categorical_features])
# Zapisujemy nazwy kolumn, aby użyć ich w skrypcie
encoded_categorical_columns = X_categorical.columns.tolist()

# 3. Łączenie
X_tabular = np.concatenate([X_numeric, X_categorical.values], axis=1)

# Podział na zbiory
X_text_train, X_text_test, X_tabular_train, X_tabular_test, y_train, y_test = train_test_split(
    X_text, X_tabular, y_categorical, test_size=0.2, random_state=42, stratify=y_categorical
)

--- Krok 1: Przygotowanie danych (typy i reguły) ---
Liczba wierszy przed balansowaniem: 135797
Rozkład klas PRZED zbalansowaniem:
BuildingCondition
DEVELOPER_STATE     135044
GOOD                   500
AFTER_RENOVATION       161
FOR_RENOVATION          92
Name: count, dtype: int64
Rozkład klas PO zbalansowaniu:
BuildingCondition
DEVELOPER_STATE     10000
GOOD                  500
AFTER_RENOVATION      161
FOR_RENOVATION         92
Name: count, dtype: int64

--- Krok 3: Uproszczone przygotowanie danych do modelu ---


In [4]:
# --- BUDOWA I TRENING MODELU ---
# Definiujemy architekturę
text_input = Input(shape=(max_len,), name='text_input')
embedding_layer = Embedding(input_dim=max_words, output_dim=128)(text_input)
lstm_layer = LSTM(64, recurrent_dropout=0.2)(embedding_layer)
dropout_lstm = Dropout(0.4)(lstm_layer)

tabular_input = Input(shape=(X_tabular_train.shape[1],), name='tabular_input')
tabular_dense = Dense(32, activation='relu')(tabular_input)

concatenated = Concatenate()([dropout_lstm, tabular_dense])
dense1 = Dense(64, activation='relu')(concatenated)
dropout_final = Dropout(0.5)(dense1)
output = Dense(len(label_mapping), activation='softmax')(dropout_final)

model = Model(inputs=[text_input, tabular_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Trening na zbalansowanych danych
print("\nRozpoczynam trening na zbalansowanych danych...")
history = model.fit(
    [X_text_train, X_tabular_train], y_train,
    epochs=10, 
    batch_size=128,
    validation_data=([X_text_test, X_tabular_test], y_test)
)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 200)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 200, 128)          │       1,280,000 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ (None, 64)                │          49,408 │ embedding[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ tabular_input (InputLayer)    │ (None, 14)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 64)                │               0 │ lstm[0][0]                 │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 32)                │             480 │ tabular_input[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 96)                │               0 │ dropout[0][0], dense[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │           6,208 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 64)                │               0 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 4)                 │             260 │ dropout_1[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,336,356 (5.10 MB)

 Trainable params: 1,336,356 (5.10 MB)

 Non-trainable params: 0 (0.00 B)


Rozpoczynam trening na zbalansowanych danych...
Epoch 1/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 19s 234ms/step - accuracy: 0.8153 - loss: 0.6615 - val_accuracy: 0.9424 - val_loss: 0.1785
Epoch 2/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 17s 248ms/step - accuracy: 0.9458 - loss: 0.1898 - val_accuracy: 0.9558 - val_loss: 0.1265
Epoch 3/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 19s 278ms/step - accuracy: 0.9580 - loss: 0.1330 - val_accuracy: 0.9642 - val_loss: 0.1077
Epoch 4/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 20s 293ms/step - accuracy: 0.9667 - loss: 0.0968 - val_accuracy: 0.9651 - val_loss: 0.1075
Epoch 5/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 21s 311ms/step - accuracy: 0.9715 - loss: 0.0830 - val_accuracy: 0.9661 - val_loss: 0.1026
Epoch 6/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 21s 306ms/step - accuracy: 0.9746 - loss: 0.0711 - val_accuracy: 0.9670 - val_loss: 0.1045
Epoch 7/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 21s 304ms/step - accuracy: 0.9736 - loss: 0.0667 - val_accuracy: 0.9623 - val_loss: 0.1213
Epoch 8/10
68/68 ━━━━━━━━━━━━━━━━━━━━ 20s 297ms/

In [5]:
import joblib
import json
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# --- ZAPIS MODELU I WSZYSTKICH POTRZEBNYCH OBIEKTÓW ---

# 1. Zapis modelu Keras
model.save('model_lstm_stan.keras')
print("Model Keras zapisany jako 'model_lstm_stan.keras'")

# 2. Zapis Tokenizera
with open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer.to_json(), ensure_ascii=False))
print("Tokenizer zapisany jako 'tokenizer.json'")

# 3. Zapis Preprocesora (ColumnTransformer)
# Definiujemy go ponownie, aby mieć pewność
numeric_features = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'year']
categorical_features = ['BuildingType', 'OfferFrom', 'TypeOfMarket']
preprocessor_to_save = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])
# Uczymy go na tych samych danych
preprocessor_to_save.fit(final_data[numeric_features + categorical_features])
joblib.dump(preprocessor_to_save, 'preprocessor.joblib')
print("Preprocessor zapisany jako 'preprocessor.joblib'")

# 4. Zapis mapowania etykiet
with open('label_mapping.json', 'w') as f:
    label_mapping_str_keys = {str(k): v for k, v in label_mapping.items()}
    json.dump(label_mapping_str_keys, f)
print("Mapowanie etykiet zapisane jako 'label_mapping.json'")

# ---- KLUCZOWA ZMIANA: ZAPISUJEMY KOLEJNOŚĆ KOLUMN ----
columns_for_prediction = numeric_features + categorical_features
joblib.dump(columns_for_prediction, 'columns_for_prediction.joblib')
print(f"Zapisano listę i kolejność kolumn do pliku: columns_for_prediction.joblib")

Model Keras zapisany jako 'model_lstm_stan.keras'
Tokenizer zapisany jako 'tokenizer.json'
Preprocessor zapisany jako 'preprocessor.joblib'
Mapowanie etykiet zapisane jako 'label_mapping.json'
Zapisano listę i kolejność kolumn do pliku: columns_for_prediction.joblib


In [6]:
# --- OCENA MODELU ---
print("\n--- Ocena modelu na zbiorze testowym ---")
loss, accuracy = model.evaluate([X_text_test, X_tabular_test], y_test)
print(f"\nDokładność na zbiorze testowym: {accuracy:.4f}")

y_pred_proba = model.predict([X_text_test, X_tabular_test])
y_pred = np.argmax(y_pred_proba, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

print("\nRaport klasyfikacji na zbiorze testowym:")
print(classification_report(y_test_labels, y_pred, target_names=list(label_mapping.values())))


--- Ocena modelu na zbiorze testowym ---
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.9678 - loss: 0.1098

Dokładność na zbiorze testowym: 0.9665
68/68 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step

Raport klasyfikacji na zbiorze testowym:
                  precision    recall  f1-score   support

AFTER_RENOVATION       0.52      0.34      0.42        32
 DEVELOPER_STATE       0.98      1.00      0.99      2000
  FOR_RENOVATION       0.60      0.16      0.25        19
            GOOD       0.76      0.68      0.72       100

        accuracy                           0.97      2151
       macro avg       0.72      0.55      0.59      2151
    weighted avg       0.96      0.97      0.96      2151



In [10]:
# --- FINALNA PREDYKCJA I ZAPIS (WERSJA KOMPLETNA, ZAPISUJĄCA WSZYSTKO) ---
import csv
import joblib

print("\n--- Rozpoczynam predykcję na całej bazie danych ---")

df_full = df.copy() 

# ... (cały kod przygotowania danych i predykcji pozostaje bez zmian) ...
# ... (aż do tej linii) ...
df_full['Predict_State'] = predicted_conditions
print("Zakończono predykcje.")

# ... (reguły biznesowe bez zmian) ...
print("Stosuję twarde reguły biznesowe...")
df_full.loc[df_full['TypeOfMarket'] == 'pierwotny', 'Predict_State'] = 'DEVELOPER_STATE'
df_full.loc[pd.to_datetime(df_full['BuiltYear'], errors='coerce').dt.year >= 2024, 'Predict_State'] = 'DEVELOPER_STATE'

# --- KROK ZAPISU (ZAPISUJEMY WSZYSTKIE KOLUMNY) ---
output_filename = 'Data_state_LSTM_predicted_full.csv'

print(f"\nRozpoczynam zapis do pliku: {output_filename}")
print(f"Zapisuję {len(df_full)} wierszy i {len(df_full.columns)} kolumn.")

# Zapisujemy CAŁY DataFrame df_full, aby mieć pewność, że wszystkie kolumny są dostępne
df_full.to_csv(
    output_filename, 
    index=False, 
    sep=';', 
    encoding='utf-8-sig',
    quoting=csv.QUOTE_ALL 
)

print(f"\nZapisano pełny zbiór danych z predykcjami do pliku: {output_filename}")


--- Rozpoczynam predykcję na całej bazie danych ---
Zakończono predykcje.
Stosuję twarde reguły biznesowe...

Rozpoczynam zapis do pliku: Data_state_LSTM_predicted_full.csv
Zapisuję 760765 wierszy i 54 kolumn.

Zapisano pełny zbiór danych z predykcjami do pliku: Data_state_LSTM_predicted_full.csv


In [9]:
df_full.info

<bound method DataFrame.info of          SaleId OriginalId PortalId  \
0            88       NULL       14   
1            99       NULL       19   
2           115       NULL        3   
3           140       NULL       14   
4           145       NULL        3   
...         ...        ...      ...   
760760  5027512       NULL       15   
760761  5027542       NULL       15   
760762  5027544       NULL       15   
760763  5027546       NULL       15   
760764  5027680       NULL       15   

                                                    Title  \
0                     Mieszkanie trzypokojowe na sprzedaż   
1       Sprzedam mieszkanie na parterze 64.8m2 Białyst...   
2           Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki   
3                     Mieszkanie trzypokojowe na sprzedaż   
4                      3 Pokoje- 48M2-Osiedle Dziesięciny   
...                                                   ...   
760760  Dwupokojowe mieszkanie na sprzedaż: Bydgoszcz,...   
760761  Pię